# 🔭 RSG Lensing Inversion - Online Demo

**Authors:** Carmen N. Wrede, Lino P. Casu

This notebook launches an online Gradio interface for gravitational lensing analysis.

## Features
- **Ring vs Cross Classification** - Automatic morphology detection
- **Model Zoo** - 8 lens models (m=2, shear, m=3, m=4)
- **3D Geometry Visualization** - Observer → Lens → Source
- **Real Data Support** - Import CSV observation packs

---
**Run all cells to get a shareable link!**

In [ ]:
#@title 1️⃣ Setup: Clone Repository & Install Dependencies
%%capture
!git clone https://github.com/error-wtf/ssz-lensing.git 2>/dev/null || (cd ssz-lensing && git pull)
!pip install gradio numpy matplotlib

In [ ]:
#@title 2️⃣ Import Modules
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
import base64

# Add repo to path
sys.path.insert(0, '/content/ssz-lensing/src')
sys.path.insert(0, '/content/ssz-lensing')

# Import lensing modules
from models.morphology import MorphologyClassifier, Morphology
from models.ring_analysis import RingAnalyzer, generate_ring_points
from geometry.triad_scene import TriadScene
from geometry.projection import project_to_lens_plane

print("✅ Modules loaded successfully!")

In [ ]:
#@title 3️⃣ Define Lensing Functions

def analyze_positions(positions_text, morphology_mode="auto"):
    """Analyze image positions."""
    try:
        # Parse positions
        lines = [l.strip() for l in positions_text.strip().split('\n') if l.strip()]
        positions = []
        for line in lines:
            parts = line.replace(',', ' ').split()
            if len(parts) >= 2:
                positions.append([float(parts[0]), float(parts[1])])
        
        if len(positions) < 2:
            return "Error: Need at least 2 positions", None
        
        positions = np.array(positions)
        
        # Classify morphology
        classifier = MorphologyClassifier()
        morph = classifier.classify(positions)
        
        # Ring analysis
        analyzer = RingAnalyzer()
        ring = analyzer.fit_ring(positions)
        
        # Generate report
        report = f"""# 🔭 Lensing Analysis Report

## Morphology Classification
- **Primary:** {morph.primary.value.upper()}
- **Confidence:** {morph.confidence:.1%}
- **Recommended Models:** {', '.join(morph.recommended_models)}

## Geometric Analysis
- **Mean Radius:** {morph.mean_radius:.4f} arcsec
- **Radial Scatter:** {morph.radial_scatter:.4f} (σ/r)
- **Azimuthal Coverage:** {morph.azimuthal_coverage:.1%}

## Harmonic Components
- **m=2 (Quadrupole):** {morph.m2_amplitude:.4f}
- **m=4 (Hexadecapole):** {morph.m4_amplitude:.4f}

## Ring Fit
- **Center:** ({ring.center_x:.4f}, {ring.center_y:.4f})
- **Radius:** {ring.radius:.4f} arcsec
- **RMS Residual:** {ring.rms_residual:.4f}
- **Perturbation:** {ring.perturbation_type}

## Notes
"""
        for note in morph.notes:
            report += f"- {note}\n"
        
        # Generate plot
        fig = create_analysis_plot(positions, morph, ring)
        
        return report, fig
    
    except Exception as e:
        return f"Error: {str(e)}", None


def create_analysis_plot(positions, morph, ring):
    """Create analysis visualization."""
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # 1. Image Plane
    ax1 = axes[0]
    ax1.scatter(positions[:, 0], positions[:, 1], c='red', s=100, zorder=5)
    for i, pos in enumerate(positions):
        ax1.annotate(f'{i+1}', (pos[0]+0.05, pos[1]+0.05), fontsize=10)
    
    # Einstein ring
    theta = np.linspace(0, 2*np.pi, 100)
    ax1.plot(ring.center_x + ring.radius*np.cos(theta),
             ring.center_y + ring.radius*np.sin(theta),
             'b--', alpha=0.5, label=f'R={ring.radius:.3f}')
    ax1.scatter([ring.center_x], [ring.center_y], c='blue', s=50, marker='+')
    ax1.set_aspect('equal')
    ax1.set_title(f'Image Plane: {morph.primary.value.upper()}')
    ax1.set_xlabel('θ_x (arcsec)')
    ax1.set_ylabel('θ_y (arcsec)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Radial Residuals
    ax2 = axes[1]
    phi = ring.azimuthal_angles
    dr = ring.radial_residuals
    idx = np.argsort(phi)
    ax2.scatter(np.degrees(phi[idx]), dr[idx], c='blue', s=50)
    ax2.axhline(0, color='gray', linestyle='--')
    
    # Harmonic models
    phi_model = np.linspace(-np.pi, np.pi, 100)
    m2 = ring.m2_component[0] * np.cos(2*phi_model - ring.m2_component[1])
    m4 = ring.m4_component[0] * np.cos(4*phi_model - ring.m4_component[1])
    ax2.plot(np.degrees(phi_model), m2, 'g-', alpha=0.7, label=f'm=2: {ring.m2_component[0]:.3f}')
    ax2.plot(np.degrees(phi_model), m4, 'orange', alpha=0.7, label=f'm=4: {ring.m4_component[0]:.3f}')
    
    ax2.set_xlabel('Angle φ (degrees)')
    ax2.set_ylabel('Δr (residual)')
    ax2.set_title('Radial Residual vs Angle')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Harmonic Signature
    ax3 = axes[2]
    bars = ax3.bar(['m=2', 'm=4'], 
                   [ring.m2_component[0], ring.m4_component[0]],
                   color=['green', 'orange'], alpha=0.7)
    ax3.axhline(0.02 * ring.radius, color='red', linestyle='--', label='2% threshold')
    ax3.set_ylabel('Amplitude')
    ax3.set_title(f'Perturbation: {ring.perturbation_type}')
    ax3.legend()
    
    plt.tight_layout()
    return fig


def generate_synthetic(system_type, n_points, noise, c2, c4):
    """Generate synthetic lensing data."""
    try:
        if system_type == "Ring (Einstein Ring)":
            positions = generate_ring_points(
                theta_E=1.0, n_points=int(n_points),
                c2=c2, s2=0, c4=c4, s4=0,
                noise=noise
            )
        elif system_type == "Quad (Einstein Cross)":
            # 4 images at typical quad positions
            angles = np.array([0.3, 1.8, 3.5, 5.2])
            r = 1.0 + c2*np.cos(2*angles) + c4*np.cos(4*angles)
            positions = np.column_stack([r*np.cos(angles), r*np.sin(angles)])
            positions += np.random.normal(0, noise, positions.shape)
        else:  # Arc
            phi = np.linspace(0.5, 2.5, int(n_points))
            r = 1.0 + c2*np.cos(2*phi) + c4*np.cos(4*phi)
            positions = np.column_stack([r*np.cos(phi), r*np.sin(phi)])
            positions += np.random.normal(0, noise, positions.shape)
        
        # Format as text
        text = "\n".join([f"{p[0]:.4f}, {p[1]:.4f}" for p in positions])
        return text
    
    except Exception as e:
        return f"Error: {str(e)}"

print("✅ Functions defined!")

In [ ]:
#@title 4️⃣ Launch Gradio Interface (with Online Link)
import gradio as gr

# Example data
EXAMPLE_QUAD = """0.740, 0.565
-0.635, 0.470
-0.480, -0.755
0.870, -0.195"""

EXAMPLE_RING = "\n".join([f"{np.cos(a):.4f}, {np.sin(a):.4f}" 
                          for a in np.linspace(0, 2*np.pi, 20, endpoint=False)])

# Build interface
with gr.Blocks(title="RSG Lensing Inversion", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🔭 RSG Lensing Inversion Framework
    **Gravitational Lensing Analysis Tool**
    
    Analyze Einstein Rings, Crosses, and Arcs with automatic morphology classification.
    """)
    
    with gr.Tabs():
        # Tab 1: Analyze
        with gr.Tab("📊 Analyze Positions"):
            with gr.Row():
                with gr.Column():
                    positions_input = gr.Textbox(
                        label="Image Positions (x, y per line)",
                        placeholder="0.740, 0.565\n-0.635, 0.470\n...",
                        lines=10,
                        value=EXAMPLE_QUAD
                    )
                    with gr.Row():
                        analyze_btn = gr.Button("🔍 Analyze", variant="primary")
                        clear_btn = gr.Button("Clear")
                    
                    gr.Examples(
                        examples=[
                            [EXAMPLE_QUAD],
                            [EXAMPLE_RING]
                        ],
                        inputs=[positions_input],
                        label="Examples"
                    )
                
                with gr.Column():
                    report_output = gr.Markdown(label="Analysis Report")
            
            plot_output = gr.Plot(label="Visualization")
            
            analyze_btn.click(
                fn=analyze_positions,
                inputs=[positions_input],
                outputs=[report_output, plot_output]
            )
            clear_btn.click(lambda: ("", None), outputs=[report_output, plot_output])
        
        # Tab 2: Generate Synthetic
        with gr.Tab("🎲 Generate Synthetic"):
            gr.Markdown("Generate synthetic lensing configurations for testing.")
            
            with gr.Row():
                system_type = gr.Dropdown(
                    choices=["Ring (Einstein Ring)", "Quad (Einstein Cross)", "Arc (Partial Ring)"],
                    value="Ring (Einstein Ring)",
                    label="System Type"
                )
                n_points = gr.Slider(4, 100, value=30, step=1, label="Number of Points")
            
            with gr.Row():
                noise = gr.Slider(0, 0.1, value=0.01, step=0.005, label="Noise Level")
                c2 = gr.Slider(0, 0.3, value=0, step=0.01, label="m=2 Perturbation")
                c4 = gr.Slider(0, 0.2, value=0, step=0.01, label="m=4 Perturbation")
            
            generate_btn = gr.Button("🎲 Generate", variant="primary")
            synthetic_output = gr.Textbox(label="Generated Positions", lines=10)
            
            generate_btn.click(
                fn=generate_synthetic,
                inputs=[system_type, n_points, noise, c2, c4],
                outputs=[synthetic_output]
            )
        
        # Tab 3: About
        with gr.Tab("ℹ️ About"):
            gr.Markdown("""
            ## RSG Lensing Inversion Framework
            
            **Authors:** Carmen N. Wrede, Lino P. Casu
            
            ### Morphology Types
            - **RING**: Einstein Ring (rotationally symmetric)
            - **QUAD**: Einstein Cross (4 discrete images)
            - **ARC**: Partial ring / giant arcs
            - **DOUBLE**: Two-image system
            
            ### Model Zoo
            - **m=2**: Quadrupole (basic ellipticity)
            - **m=2 + shear**: External tidal field
            - **m=2 + m=3**: Octupole (asymmetry)
            - **m=2 + m=4**: Hexadecapole (boxiness)
            - All combinations with m=4
            
            ### Harmonic Analysis
            The radial residuals Δr(φ) are decomposed:
            - `cos(2φ)` → Quadrupole/Shear (elliptical)
            - `cos(4φ)` → Boxiness/Diskiness
            
            ### References
            - GitHub: [ssz-lensing](https://github.com/error-wtf/ssz-lensing)
            """)

# Launch with share=True for public link
print("\n" + "="*60)
print("🚀 Launching Gradio Interface with PUBLIC LINK...")
print("="*60 + "\n")

demo.launch(share=True, debug=False)